In [1]:
import lightkurve as lk

In [3]:
'''
Target Pixel Files (TPFs) are a file common to Kepler/K2 and the TESS mission. They contain movies of the pixel data centered on a single target star.

TPFs can be thought of as stacks of images, with one image for every timestamp the telescope took data. Each timestamp is referred to as a cadence. 
These images are cut out "postage stamps" of the full observation to make them easier to work with.

TPFs are given in FITS files, LightKurve includes tools that allow us to directly work with these files.
'''

'\nTarget Pixel Files (TPFs) are a file common to Kepler/K2 and the TESS mission. They contain movies of the pixel data centered on a single target star.\n\nTPFs can be thought of as stacks of images, with one image for every timestamp the telescope took data. Each timestamp is referred to as a cadence. \nThese images are cut out "postage stamps" of the full observation to make them easier to work with.\n\nTPFs are given in FITS files, LightKurve includes tools that allow us to directly work with these files.\n'

In [4]:
'''
To load a KeplerTargetPixelFile from a local path or remote url, simply call Lightkurves read function using the location of the file as the parameter:
'''

#tpf is an object of type target_pixle_file, this object contains easy accsess metadata (mission title, quarter...)
tpf = lk.read("https://archive.stsci.edu/pub/kepler/target_pixel_files/0069/006922244/kplr006922244-2010078095331_lpd-targ.fits.gz")



